In [2]:
import heapq
import folium 
from folium.plugins import HeatMap

class NetworkNode:
    def __init__(self, ID, latitude, longitude):
        self.ID = ID
        self.latitude = latitude
        self.longitude = longitude

    def __repr__(self):
        return f"{self.__class__.__name__}({self.ID})"

    def __hash__(self):
        return hash(self.ID)

    def __eq__(self, other):
        return (self.ID == other.ID) and(type(self) is type(other))

# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
class FogCloudNode(NetworkNode):
    def __init__(self, ID, longitude, latitude, processing_capacity, memory_capacity, cost, model):
        super().__init__(ID, latitude, longitude)
        self.model = model
        self.processing_capacity = processing_capacity
        self.memory_capacity = memory_capacity
        self.cost = cost
        self.requisicoes = 0

class FogNode(FogCloudNode):
    pass

class CloudNode(FogCloudNode):
    pass

# Sensores -> index | longitude | latitude | services
class Sensor(NetworkNode):
    def __init__(self, ID, longitude, latitude, services):
        super().__init__(ID, latitude, longitude)
        self.services = services

class Grafo:
    def __init__(self):
        self.adj = {}
        self.adj_ = {}
        self.n_vertices = 0
        self.n_arestas = 0

    def add_vertice(self, noh):
        if noh not in self.adj:
            self.adj[noh] = []
            self.n_vertices += 1

    def add_aresta(self, u, v, largura_banda, custo, dist_H):
        self.add_vertice(u)
        self.add_vertice(v)
        # Tempo de delay = latência (valor obtido da tecnologia) + propagação (harvesine/2*10^8)
        tempo = 0.001 + float(dist_H)/(2*10**8)
        self.adj[u].append((v, largura_banda, custo, tempo))
        self.adj_[f"({u},{v})"] = float(largura_banda)*10**9
        self.n_arestas += 1

    def copiar(self):
        novo = Grafo()
        novo.n_arestas =  self.n_arestas
        for vertice, arestas in self.adj.items():
            novo.add_vertice(vertice)
            for v, largura_banda, custo, tempo in arestas:
                novo.adj[vertice].append((v, largura_banda, custo, tempo))
        return novo

def dijkstra(grafo, origem):
    tempo = {n: float('inf') for n in grafo.adj}
    tempo[origem] = 0
    prev = {n: None for n in grafo.adj}
    fila = [(0, origem)]

    while fila:
        tempo_atual, u = heapq.heappop(fila)
        if tempo_atual > tempo[u]:
            continue

        for v, _, _, t in grafo.adj[u]:
            novo_tempo = tempo[u] + t 
            if novo_tempo < tempo[v]:
                tempo[v] = novo_tempo
                prev[v] = u
                heapq.heappush(fila, (novo_tempo, v))

    return tempo, prev

def reconstruir_caminho(prev, destino):
    caminho = []
    while destino is not None:
        caminho.append(destino)
        destino = prev[destino]
    return caminho[::-1]

def processar_caminho(grafo, caminho, requisicao, instante):
    arcos = []
    band_tot = 0
    custo = 0
    selecionado = None
    for i in range(len(caminho)-1):
        vertice_atual = caminho[i]
        vizinhos = grafo.adj[vertice_atual]
        # Selecionar o vizinho correto
        # u: [v, largura_banda, custo, tempo]
        for vizinho in vizinhos:
            if(vizinho[0] == caminho[i+1]) and (requisicao[2] <= grafo.adj_[f"({vertice_atual},{vizinho[0]})"]):
                selecionado = vizinho[0]
                arcos.append(f"({vertice_atual},{selecionado})")
                grafo.adj_[f"({vertice_atual},{selecionado})"] -= requisicao[2]
                temporal_arestas[instante].append((f"({vertice_atual},{selecionado})", requisicao[2]))
                band_tot += float(vizinho[1])
                custo += float(vizinho[1])*float(vizinho[2])
                break
        if selecionado is None:
            return selecionado, False, arcos, band_tot, custo
        if(requisicao[0] <= selecionado.processing_capacity):
            if(requisicao[1] <= selecionado.memory_capacity):
                selecionado.processing_capacity -= requisicao[0]
                selecionado.memory_capacity -= requisicao[1]
                selecionado.requisicoes += 1
                return selecionado, True, arcos, band_tot, custo
    return selecionado, False, arcos, band_tot, custo   
        

# Próximos passos - leitura da instância na main
# Leitura e armazenamentos dos dados até "#end_services"
    # instance_info -> cabeçalho (apenas algumas informações são importantes
    # sensors -> ler e armazenar numa lista de objetos Sensor()
    # reach_fog_nodes -> ler e armazenar num grafo as arestas Sensor-Fog
    # fog -> ler e armazenar numa lista de objetos FogNode() e arestas Fog-Fog
    # cloud -> ler e armazenar numa lista de objetos CloudNode() e arestas Fog-Cloud
    # service -> ler e armazenar num dicionário de services, com a chave sendo o nome do serviço
##
# Começa a leitura das requests
    # Para cada time_instant
        # Armazena o valor do time instant numa variável i
        # Verifica na ED temporal se naquele i precisa de alteração no grafo (retorno de disponibilidade de capacidade)
        # Se sim, realiza todas as alterações necessárias
        # Roda o Dijkstra
        # Começa a percorrer as requisições do tipo "sensor ID | service index | type of service | request_lifetime"
        # Chama a função processar_caminho() para essa requisição
        # Se True
            # ED Temporal: adiciona para o instante i+request_lifetime+1 o valor (FogCloudNode, services[type_of_service][0], services[type_of_service][1])
            # Numero de requisições aceitas: incrementado
            # Número de requisições do tipo "type of service" aceitas: incrementado
            # Dentro de processar_caminho()
                # Sinaliza o nó fog em que foi aceita a requisição
                # Conta a quantidade de arcos utilizados do sensor inicial até o nó fog em que foi processado
                # Soma a quantidade de largura de banda 
                # Soma o custo dos arcos com o custo do processamento
                # Retorna todos esse valores
# Após o "#end_requests", retorna todos os valores de dados de análise e gera-se o mapa de calor 

with open("1.txt", "r", encoding="utf-8") as f:
    grafo = Grafo()
    for i in range(7):
        f.readline()
    linha = f.readline().strip()
    num_sensores = int(linha.split()[1])
    linha = f.readline().strip()
    num_fog = int(linha.split()[1])
    linha = f.readline().strip()
    num_cloud = int(linha.split()[1])
    linha = f.readline().strip()
    num_servicos = int(linha.split()[1])
    f.readline()
    f.readline()
    sensores = []
    for i in range(num_sensores):
        linha = f.readline().strip().split()
        # Sensores -> index | longitude | latitude | services
        sensor = Sensor(linha[0], linha[1], linha[2], linha[3::])
        sensores.append(sensor)
        grafo.add_vertice(sensor)
    f.readline()
    f.readline()
    fogs = []
    for i in range(num_fog):
        linha = f.readline().strip().split()
        # Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
        fog = FogNode(linha[0], float(linha[1]), float(linha[2]), float(linha[3]), float(linha[4]), float(linha[5]), linha[6])
        fogs.append(fog)
        grafo.add_vertice(fog)
    num_fogfog = int(f.readline().strip())
    for i in range(num_fogfog):
        linha = f.readline().strip().split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(fogs[int(linha[1])], fogs[int(linha[2])], linha[3], linha[4], linha[5])
    f.readline()
    f.readline()
    linha = f.readline().strip()
    while linha != "#end_reach_fog_nodes":
        linha = linha.split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(sensores[int(linha[1])], fogs[int(linha[2])], linha[3], linha[4], linha[5])
        linha = f.readline().strip()
    f.readline()
    clouds = []
    for i in range(num_cloud):
        linha = f.readline().strip().split()
        # Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
        cloud = CloudNode(linha[0], float(linha[1]), float(linha[2]), float(linha[3]), float(linha[4]), float(linha[5]), linha[6])
        clouds.append(cloud)
    num_fogcloud = int(f.readline().strip())
    for i in range(num_fogcloud):
        linha = f.readline().strip().split()
        # Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
        grafo.add_aresta(fogs[int(linha[1])], clouds[int(linha[2])], linha[3], linha[4], linha[5])
    f.readline()
    f.readline()
    servicos = {}
    for i in range(num_servicos):
        linha =  f.readline().strip().split()
        # ID: [processing_demand, memory_demand, number_of_bits, lifetime]
        servicos[linha[0]] = [float(linha[1]), float(linha[2]), int(linha[3]), int(linha[4])]
    f.readline() # #end_services
    f.readline() # #begin_requests
    ## PARTE 2 - Leitura das requests
    temporal = {i: [] for i in range(1, 1001)}
    temporal_arestas = {i: [] for i in range(1, 1001)}
    linha = f.readline().strip()
    tot_req = 0
    quant_req = 0
    set_arcos = []
    quant_band = 0
    quant_custo = 0
    while linha != "#end_requests":
        instante = int(linha.split("_")[2])
        for (v, p, m) in temporal[instante]:
            v.processing_capacity += p
            v.memory_capacity += m
        for aresta, band in temporal_arestas[instante]:
            grafo.adj_[aresta] += band
        linha =  f.readline().strip()
        while (linha.split("_")[0] != "##time") and (linha != "#end_requests"):
            linha = linha.split()
            # sensor ID | service index | type of service | request_lifetime
            tot_req += 1
            _, prev = dijkstra(grafo, sensores[int(linha[0])])
            caminho = reconstruir_caminho(prev, clouds[0])
            selecionado, processou, arcos, band, c = processar_caminho(grafo, caminho, servicos[linha[2]], instante)
            if processou:
                quant_req +=1
                set_arcos += arcos
                quant_band += band
                quant_custo += c
                #tX: [(Nó, processing_demand, memory_demand)]
                if (instante+int(linha[3])+1) <= 1000:
                    temporal[instante+int(linha[3])+1].append((selecionado, servicos[linha[2]][0], servicos[linha[2]][1]))
            linha = f.readline().strip()
    set_arcos = set(set_arcos)
    quant_arcos = len(set_arcos)
print(f"Total de requisições: {tot_req}\n % de requisições processadas: {(quant_req/tot_req)*100.0}%\n % de arcos usados: {(quant_arcos/grafo.n_arestas)*100.0}%\n Largura de banda usada: {quant_band}\n Custo gasto: {quant_custo}\n")

# Inicializa o mapa centralizado na média das coordenadas
avg_lat = sum(node.latitude for node in fogs) / len(fogs)
avg_lon = sum(node.longitude for node in fogs) / len(fogs)
mapa = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

# Constrói os dados para o mapa de calor
heat_data = [
    [node.latitude, node.longitude, node.requisicoes]
    for node in fogs if node.requisicoes > 0
]

# Adiciona camada de calor
HeatMap(heat_data).add_to(mapa)

# Adiciona marcadores (opcional)
for node in fogs:
    folium.Marker(
        location=[node.latitude, node.longitude],
        popup=f"{node.ID} - {node.requisicoes} reqs",
        icon=folium.Icon(color="blue" if node.requisicoes > 0 else "gray")
    ).add_to(mapa)

# Salvar como HTML interativo
mapa.save("mapa_calor.html")

'''# Criando os nós
# Sensores -> index | longitude | latitude | services
# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
sensor = Sensor("S1", 0.0, 0.0, services=[])
fog1 = FogNode("F1", 1.0, 1.0, 80, 16, 5, "model-A")
fog2 = FogNode("F2", 1.5, 1.5, 70, 8, 6, "model-B")
cloud = CloudNode("C1", 2.0, 2.0, 500, 128, 15, "cloud-X")

# Criando o grafo e adicionando conexões
g = Grafo()

# Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
g.add_aresta(sensor, fog1, 3, 4, 12)     
g.add_aresta(sensor, fog2, 5, 6, 30)     
g.add_aresta(fog1, fog2, 4, 5, 40)       
g.add_aresta(fog2, fog1, 6, 5, 12)      
g.add_aresta(fog1, cloud, 4, 8, 90)      
g.add_aresta(fog2, cloud, 4, 4, 120)      

# ID: [processing_demand, memory_demand, number_of_bits, lifetime]
services = {
    'waste': [0.2125, 0.375, 296, 50],
    'camera': [0.35, 0.475, 12000, 10],
    'air': [0.25, 0.3125, 744, 100]
}
tempo, prev = dijkstra(g, sensor)
print(tempo)
print(prev)
caminho = reconstruir_caminho(prev, cloud)
print(caminho)
print(g.adj)
print("#"*30)
print(processar_caminho(g.copiar(), caminho, services["waste"]))'''


Total de requisições: 5892
 % de requisições processadas: 79.29395790902919%
 % de arcos usados: 29.230769230769234%
 Largura de banda usada: 58500.72000000224
 Custo gasto: 4852.754096000328



'# Criando os nós\n# Sensores -> index | longitude | latitude | services\n# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model\n# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model\nsensor = Sensor("S1", 0.0, 0.0, services=[])\nfog1 = FogNode("F1", 1.0, 1.0, 80, 16, 5, "model-A")\nfog2 = FogNode("F2", 1.5, 1.5, 70, 8, 6, "model-B")\ncloud = CloudNode("C1", 2.0, 2.0, 500, 128, 15, "cloud-X")\n\n# Criando o grafo e adicionando conexões\ng = Grafo()\n\n# Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j\ng.add_aresta(sensor, fog1, 3, 4, 12)     \ng.add_aresta(sensor, fog2, 5, 6, 30)     \ng.add_aresta(fog1, fog2, 4, 5, 40)       \ng.add_aresta(fog2, fog1, 6, 5, 12)      \ng.add_aresta(fog1, cloud, 4, 8, 90)      \ng.add_aresta(fog2, cloud, 4, 4, 120)      \n\n# ID: [processing_demand, memory_demand, number_of_bits, lifetime]\nservices = {\n    \'waste\': [0.2125,